In [1]:
## Import dependencies

import numpy as np
from os import path
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import time


# Set default plot size
plt.rcParams["figure.figsize"] = (30,20)

# Define number of epochs used later in training
num_epochs = 500

# Identification part of the filenames
model_base_name = 'LogCombined1Distribution'
dataset_base_name = 'Combined1Distribution'

## Train CNN on dataset of 44 metabolites

In [2]:
# Name variable used for saving model metrics, name should reflect model used, dataset used, and other information such as # of epochs
ModelName = f"CNN_44met_{model_base_name}Dist_TrainingAndValidation_ForManuscript_" + str(num_epochs) +"ep"

# Set the random seed
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/ModelPerformanceMetrics/') 
seed = 1
torch.manual_seed(seed)
np.save(ModelName + "_Seed.npy", seed)

In [3]:
## Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load training data and max value from testing and training datasets
spectra = np.load(f'Dataset44_{dataset_base_name}_ForManuscript_Spec.npy')
conc1 = np.load(f'Dataset44_{dataset_base_name}_ForManuscript_Conc.npy')


# Transform data using log transformation to compress dynamic range
b = 1000  # compression constant
spectra = np.log10(1+spectra*b)   # perform transformation
OvMax = spectra.max()   # determine max value, for scaling entire range from 0 to 1
spectra = spectra/OvMax   # scale data
print(OvMax)   

3.000434077479319


In [4]:
## Prepare to switch data from CPU to GPU

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    device = torch.device("cuda")          # A CUDA device object
    print("Using GPU for training.")
else:
    device = torch.device("cpu")           # A CPU object
    print("CUDA is not available. Using CPU for training.")

Using GPU for training.


In [5]:
## Set up data for testing and training

# Split into testing and training data
X_train, X_test, y_train, y_test = train_test_split(spectra, conc1, test_size = 0.2, random_state = 1)

# Tensorize and prepare datasets
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


# Move the input data to the GPU device
X_train = X_train.to(device)
X_test = X_test.to(device)
#spectraVal = torch.tensor(spectraVal).float().to(device)   # Confusing names, these spectra are the 5000 spectra generated like the training dataset
#ValSpectra = torch.tensor(ValSpectra).float().to(device)   # Confusing names, these spectra are the 10 representative example spectra

# Move the target data to the GPU device
y_train = y_train.to(device)
y_test = y_test.to(device)
#concVal = torch.tensor(concVal).float().to(device)
#ValConc = torch.tensor(ValConc).float().to(device)

# More data prep?
datasets = torch.utils.data.TensorDataset(X_train, y_train)
Test_datasets = torch.utils.data.TensorDataset(X_test, y_test)
train_dataset_reshaped = [(data.unsqueeze(1), label) for data, label in datasets]
test_dataset_reshaped = [(data.unsqueeze(1), label) for data, label in Test_datasets]
train_iter = torch.utils.data.DataLoader(train_dataset_reshaped, batch_size = 64, shuffle=True)
test_iter = torch.utils.data.DataLoader(test_dataset_reshaped, batch_size = 64, shuffle=True)

In [6]:
del X_train
del X_test
del y_train
del y_test
del spectra
del conc1
del datasets
del Test_datasets

In [7]:
## Define NN model object, define some parameters, and instantiate model

# Define some model & training parameters
criterion = nn.MSELoss(reduction='sum')


# Define model
class NMR_Model_Aq(nn.Module):
    def __init__(self):
        super(NMR_Model_Aq, self).__init__()
        self.conv1 = nn.Conv1d(1, 42, kernel_size=6, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv2 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv3 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.conv4 = nn.Conv1d(42, 42, kernel_size=6, padding=1)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(120708, 200)
        self.fc2 = nn.Linear(200, 44)

    def forward(self, x):
        x = x.permute(0, 2, 1)                  
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

    

In [8]:
class RelativeAbsoluteError(nn.Module):
    def __init__(self):
        super(RelativeAbsoluteError, self).__init__()

    def forward(self, y_pred, y_true):
        # Compute the mean of the true values
        y_mean = torch.mean(y_true)
        
        # Compute the absolute differences
        absolute_errors = torch.abs(y_true - y_pred)
        mean_absolute_errors = torch.abs(y_true - y_mean)
        
        # Compute RAE
        rae = torch.sum(absolute_errors) / torch.sum(mean_absolute_errors)
        return rae

In [9]:
def train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path):
    criterion = RelativeAbsoluteError()
    optimizer = optim.AdamW(model.parameters(), lr = 5.287243368897864e-05, weight_decay=0.01)
    
    
    train_losses = []
    test_losses = []
    best_test_loss = float('inf')
    epochs_no_improve = 0
    patience = 50  # Set how many epochs without improvement in validation loss constitutes early stopping

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        train_losses.append(train_loss)

        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item() * inputs.size(0)
            test_losses.append(test_loss)
            
            
        if (epoch + 1) % 1 == 0:  # The last number here denotes how often to print loss metrics in terms of epochs
            print(f'Epoch [{epoch + 1}/{num_epochs}], '
                  f'Train Loss: {train_loss:.4f}, '
                  f'Test Loss: {test_loss:.4f}')
            
    
            
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            epochs_no_improve = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path)
        else:
            epochs_no_improve += 1
            
        if epochs_no_improve >= patience:
            print(f'Early stopping at epoch {epoch + 1}')
            break



    return train_losses, test_losses


def train_or_load_model(model, train_loader, test_loader, num_epochs, save_path):
    train_losses = []
    test_losses = []
    is_model_trained = False  # Initialize flag

    if os.path.isfile(save_path):
        print("Loading pretrained model from {}".format(save_path))
        checkpoint = torch.load(save_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters())  
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
    
    else:
        print("No pretrained model found. Training from scratch.")
        #optimizer = optim.Adam(model.parameters())  
        train_losses, test_losses = train_and_save_best_model(model, train_loader, test_loader, num_epochs, save_path)
        is_model_trained = True  # Set flag to True after training
        # Save losses per epoch
        np.save(ModelName + "_TrainLoss.npy", train_losses)
        np.save(ModelName + "_TestLoss.npy", test_losses)
    
    return train_losses, test_losses, is_model_trained  # Return the losses and flag

In [10]:
## Instantiate model and train

# For timing cell run time
start_time = time.time()


# Switch to directory for saving model parameters
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Create model
model_aq = NMR_Model_Aq()

# Move the model to the GPU device
model_aq.to(device)

# Define the path to save and load the model parameters
save_path = ModelName + '_Params.pt'

# Call the function
train_losses, test_losses, is_model_trained = train_or_load_model(model_aq, train_iter, test_iter, num_epochs, save_path)


# Finish timing cell run time
end_time = time.time()
execution_time = end_time - start_time
if is_model_trained:
    np.save(ModelName + "_ExecutionTime.npy", execution_time)
    print("Execution time:", execution_time, "seconds")

No pretrained model found. Training from scratch.
Epoch [1/500], Train Loss: 9575.2832, Test Loss: 2093.3671
Epoch [2/500], Train Loss: 8128.7083, Test Loss: 1922.8638
Epoch [3/500], Train Loss: 6928.0221, Test Loss: 1485.8797
Epoch [4/500], Train Loss: 4995.0037, Test Loss: 995.7652
Epoch [5/500], Train Loss: 3312.4544, Test Loss: 673.1644
Epoch [6/500], Train Loss: 2335.2785, Test Loss: 506.9321
Epoch [7/500], Train Loss: 1786.9338, Test Loss: 388.3933
Epoch [8/500], Train Loss: 1466.4168, Test Loss: 328.7259
Epoch [9/500], Train Loss: 1253.0833, Test Loss: 311.1136
Epoch [10/500], Train Loss: 1120.3570, Test Loss: 255.6960
Epoch [11/500], Train Loss: 1018.4177, Test Loss: 235.0803
Epoch [12/500], Train Loss: 925.4401, Test Loss: 217.2285
Epoch [13/500], Train Loss: 838.4095, Test Loss: 196.9641
Epoch [14/500], Train Loss: 791.6975, Test Loss: 190.1522
Epoch [15/500], Train Loss: 747.6516, Test Loss: 184.5584
Epoch [16/500], Train Loss: 688.7650, Test Loss: 167.7627
Epoch [17/500], T

Epoch [142/500], Train Loss: 299.7413, Test Loss: 80.6477
Epoch [143/500], Train Loss: 301.8026, Test Loss: 82.9965
Epoch [144/500], Train Loss: 299.2102, Test Loss: 75.3528
Epoch [145/500], Train Loss: 300.8804, Test Loss: 71.1420
Epoch [146/500], Train Loss: 308.3581, Test Loss: 95.4787
Epoch [147/500], Train Loss: 305.5379, Test Loss: 74.7687
Epoch [148/500], Train Loss: 287.2293, Test Loss: 71.4391
Epoch [149/500], Train Loss: 307.0277, Test Loss: 87.3934
Epoch [150/500], Train Loss: 298.0129, Test Loss: 66.0297
Epoch [151/500], Train Loss: 288.7181, Test Loss: 81.9144
Epoch [152/500], Train Loss: 288.0057, Test Loss: 79.7522
Epoch [153/500], Train Loss: 286.9550, Test Loss: 69.4918
Epoch [154/500], Train Loss: 295.4769, Test Loss: 76.7155
Epoch [155/500], Train Loss: 282.0476, Test Loss: 65.0772
Epoch [156/500], Train Loss: 289.4708, Test Loss: 80.8700
Epoch [157/500], Train Loss: 288.9826, Test Loss: 74.6350
Epoch [158/500], Train Loss: 302.6633, Test Loss: 71.0647
Epoch [159/500

Epoch [284/500], Train Loss: 230.4500, Test Loss: 58.8641
Epoch [285/500], Train Loss: 237.4897, Test Loss: 56.2557
Epoch [286/500], Train Loss: 229.1374, Test Loss: 67.6794
Epoch [287/500], Train Loss: 228.6268, Test Loss: 55.5350
Epoch [288/500], Train Loss: 230.6388, Test Loss: 60.5606
Epoch [289/500], Train Loss: 226.6568, Test Loss: 66.4525
Epoch [290/500], Train Loss: 232.0678, Test Loss: 60.0074
Epoch [291/500], Train Loss: 228.8946, Test Loss: 67.8339
Epoch [292/500], Train Loss: 236.0785, Test Loss: 62.2503
Epoch [293/500], Train Loss: 229.0900, Test Loss: 66.5998
Epoch [294/500], Train Loss: 225.5359, Test Loss: 66.1668
Epoch [295/500], Train Loss: 227.9993, Test Loss: 61.3800
Epoch [296/500], Train Loss: 228.2985, Test Loss: 61.0704
Epoch [297/500], Train Loss: 229.4551, Test Loss: 64.9052
Epoch [298/500], Train Loss: 232.0487, Test Loss: 63.7802
Epoch [299/500], Train Loss: 227.4013, Test Loss: 62.0337
Epoch [300/500], Train Loss: 227.5436, Test Loss: 58.7397
Epoch [301/500

In [11]:
np.array(test_losses).min()

55.09879118204117

In [12]:
## Load training and testing datasets, validation datasets, and representative example spectra 

# Switch to directory containing datasets
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/GeneratedDataAndVariables')

# Load validation dataset
spectraVal = np.load(f'Dataset44_{dataset_base_name}_ForManuscript_Val_Spec.npy')
spectraVal = np.log10(1+spectraVal*b)   # perform transformation
spectraVal = spectraVal/OvMax   # scale data
concVal = np.load(f'Dataset44_{dataset_base_name}_ForManuscript_Val_Conc.npy')



# Load representative validation spectra and concentrations
# Load spectra of varied concentrations (all metabolites at X-mM from 0.005mm to 20mM)
ConcSpec = np.load(f'Concentration_44met_{dataset_base_name}_ForManuscript_Spec.npy')
ConcSpec = np.log10(1+ConcSpec*b)   # perform transformation
ConcSpec = ConcSpec/OvMax   # scale data
ConcConc = np.load(f'Concentration_44met_{dataset_base_name}_ForManuscript_Conc.npy')  
#  Load uniform concentration distribution validation spectra
UniformSpec = np.load(f'UniformDist_44met_{dataset_base_name}_ForManuscript_Spec.npy')
UniformSpec = np.log10(1+UniformSpec*b)   # perform transformation
UniformSpec = UniformSpec/OvMax   # scale data
UniformConc = np.load(f'UniformDist_44met_{dataset_base_name}_ForManuscript_Conc.npy')  
#  Load low concentration uniform concentration distribution validation spectra
LowUniformSpec = np.load(f'LowUniformDist_44met_{dataset_base_name}_ForManuscript_Spec.npy')
LowUniformSpec = np.log10(1+LowUniformSpec*b)   # perform transformation
LowUniformSpec = LowUniformSpec/OvMax   # scale data
LowUniformConc = np.load(f'LowUniformDist_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load tissue mimicking concentration distribution validation spectra
MimicTissueRangeSpec = np.load(f'MimicTissueRange_44met_{dataset_base_name}_ForManuscript_Spec.npy')
MimicTissueRangeSpec = np.log10(1+MimicTissueRangeSpec*b)   # perform transformation
MimicTissueRangeSpec = MimicTissueRangeSpec/OvMax   # scale data
MimicTissueRangeConc = np.load(f'MimicTissueRange_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load liver tissue mimicking concentration distribution (high relative glucose) validation spectra
MimicTissueRangeGlucSpec = np.load(f'MimicTissueRangeGluc_44met_{dataset_base_name}_ForManuscript_Spec.npy')
MimicTissueRangeGlucSpec = np.log10(1+MimicTissueRangeGlucSpec*b)   # perform transformation
MimicTissueRangeGlucSpec = MimicTissueRangeGlucSpec/OvMax   # scale data
MimicTissueRangeGlucConc = np.load(f'MimicTissueRangeGluc_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load high dynamic range #2 validation spectra
HighDynamicRange2Spec = np.load(f'HighDynRange2_44met_{dataset_base_name}_ForManuscript_Spec.npy')
HighDynamicRange2Spec = np.log10(1+HighDynamicRange2Spec*b)   # perform transformation
HighDynamicRange2Spec = HighDynamicRange2Spec/OvMax   # scale data
HighDynamicRange2Conc = np.load(f'HighDynRange2_44met_{dataset_base_name}_ForManuscript_Conc.npy') 
#  Load varied SNR validation spectra
SNR_Spec = np.load(f'SNR_44met_{dataset_base_name}_ForManuscript_Spec.npy')
SNR_Spec = np.log10(1+SNR_Spec*b)   # perform transformation
SNR_Spec = SNR_Spec/OvMax   # scale data
SNR_Conc = np.load(f'SNR_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load random singlet validation spectra
Singlet_Spec = np.load(f'Singlet_44met_{dataset_base_name}_ForManuscript_Spec.npy')
Singlet_Spec = np.log10(1+Singlet_Spec*b)   # perform transformation
Singlet_Spec = Singlet_Spec/OvMax   # scale data
Singlet_Conc = np.load(f'Singlet_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load random qref checker validation spectra
QrefSensSpec = np.load(f'QrefSensitivity_44met_{dataset_base_name}_ForManuscript_Spec.npy')
QrefSensSpec = np.log10(1+QrefSensSpec*b)   # perform transformation
QrefSensSpec = QrefSensSpec/OvMax   # scale data
QrefSensConc = np.load(f'QrefSensitivity_44met_{dataset_base_name}_ForManuscript_Conc.npy')
#  Load other validation spectra
OtherValSpectra = np.load(f'OtherVal_44met_{dataset_base_name}_ForManuscript_Spec.npy')
OtherValSpectra = np.log10(1+OtherValSpectra*b)   # perform transformation
OtherValSpectra = OtherValSpectra/OvMax   # scale data
OtherValConc = np.load(f'OtherVal_44met_{dataset_base_name}_ForManuscript_Conc.npy')




# Move the input data to the GPU device
spectraVal = torch.tensor(spectraVal).float().to(device)   
concVal = torch.tensor(concVal).float().to(device)
ConcSpec = torch.tensor(ConcSpec).float().to(device)   
ConcConc = torch.tensor(ConcConc).float().to(device)
UniformSpec = torch.tensor(UniformSpec).float().to(device)   
UniformConc = torch.tensor(UniformConc).float().to(device)
LowUniformSpec = torch.tensor(LowUniformSpec).float().to(device)   
LowUniformConc = torch.tensor(LowUniformConc).float().to(device)
MimicTissueRangeSpec = torch.tensor(MimicTissueRangeSpec).float().to(device)   
MimicTissueRangeConc = torch.tensor(MimicTissueRangeConc).float().to(device)
MimicTissueRangeGlucSpec = torch.tensor(MimicTissueRangeGlucSpec).float().to(device)   
MimicTissueRangeGlucConc = torch.tensor(MimicTissueRangeGlucConc).float().to(device)
HighDynamicRange2Spec = torch.tensor(HighDynamicRange2Spec).float().to(device)   
HighDynamicRange2Conc = torch.tensor(HighDynamicRange2Conc).float().to(device)
SNR_Spec = torch.tensor(SNR_Spec).float().to(device)   
SNR_Conc = torch.tensor(SNR_Conc).float().to(device)
Singlet_Spec = torch.tensor(Singlet_Spec).float().to(device)   
Singlet_Conc = torch.tensor(Singlet_Conc).float().to(device)
QrefSensSpec = torch.tensor(QrefSensSpec).float().to(device)   
QrefSensConc = torch.tensor(QrefSensConc).float().to(device)
OtherValSpectra = torch.tensor(OtherValSpectra).float().to(device)   
OtherValConc = torch.tensor(OtherValConc).float().to(device)

In [13]:
## Make sure best parameters are being utilized

# Switch to directory for saving model parameters
os.chdir('/home/htjhnson/Desktop/DL-NMR-Optimization/SavedParamsAndTrainingMetrics')

# Define the path where you saved your model parameters
save_path = ModelName + '_Params.pt'

# Load the entire dictionary from the saved file
checkpoint = torch.load(save_path)

# Instantiate the model
model_aq = NMR_Model_Aq()

# Load the model's state dictionary from the loaded dictionary
model_aq.load_state_dict(checkpoint['model_state_dict'])

# Move the model to the GPU 
model_aq.to(device)

NMR_Model_Aq(
  (conv1): Conv1d(1, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv3): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv4): Conv1d(42, 42, kernel_size=(6,), stride=(1,), padding=(1,))
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=120708, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=44, bias=True)
)

In [14]:
## Compute absolute percent error statistics on validation set

APEs = []
MAPEs = []

for i in np.arange(5000):
    GroundTruth = concVal[i].cpu().numpy()
    model_aq.eval()
    Prediction = model_aq(spectraVal[i].unsqueeze(0).unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ValExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ValExamples_MAPEs.npy", np.array(MAPEs))



print('Overall MAPE: ',np.array(MAPEs).mean())


Overall MAPE:  68.06779148222526


In [15]:
## Compute absolute percent error statistics on concentration varied validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = ConcConc[i]
    model_aq.eval()
    Prediction = model_aq(ConcSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "ConcExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "ConcExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Concentrations:",ConcConc[i][0].item())

Overall MAPE:  inf
--------------------
inf  - Concentrations: 0.0
111.17  - Concentrations: 0.004999999888241291
24.02  - Concentrations: 0.02500000037252903
6.19  - Concentrations: 0.10000000149011612
3.2  - Concentrations: 0.25
1.92  - Concentrations: 0.5
1.61  - Concentrations: 1.0
1.24  - Concentrations: 2.5
0.76  - Concentrations: 10.0
1.45  - Concentrations: 20.0


In [16]:
## Compute absolute percent error statistics on uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = UniformConc[i]
    model_aq.eval()
    Prediction = model_aq(UniformSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "UniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "UniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(UniformConc[i].min().item(),4), " - Mean Value:", np.round(UniformConc[i].mean().item(),1))

Overall MAPE:  4.362201
--------------------
1.44  - Min Value: 0.6783  - Mean Value: 9.2
26.48  - Min Value: 0.0096  - Mean Value: 10.3
1.21  - Min Value: 0.147  - Mean Value: 10.5
2.55  - Min Value: 0.5572  - Mean Value: 8.5
1.3  - Min Value: 1.3567  - Mean Value: 10.6
1.81  - Min Value: 0.6332  - Mean Value: 10.9
1.82  - Min Value: 0.7017  - Mean Value: 11.0
4.24  - Min Value: 0.3674  - Mean Value: 8.9
1.2  - Min Value: 0.8387  - Mean Value: 9.8
1.56  - Min Value: 1.0913  - Mean Value: 11.1


In [17]:
## Compute absolute percent error statistics on low concentration uniform distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = LowUniformConc[i]
    model_aq.eval()
    Prediction = model_aq(LowUniformSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "LowUniformExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "LowUniformExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(LowUniformConc[i].min().item(),4), " - Mean Value:", np.round(LowUniformConc[i].mean().item(),1))

Overall MAPE:  11.306401
--------------------
8.62  - Min Value: 0.0111  - Mean Value: 0.1
9.82  - Min Value: 0.0103  - Mean Value: 0.1
7.77  - Min Value: 0.0153  - Mean Value: 0.1
9.62  - Min Value: 0.0117  - Mean Value: 0.1
14.97  - Min Value: 0.0089  - Mean Value: 0.1
18.24  - Min Value: 0.0075  - Mean Value: 0.1
12.0  - Min Value: 0.0117  - Mean Value: 0.1
11.92  - Min Value: 0.0052  - Mean Value: 0.1
10.75  - Min Value: 0.008  - Mean Value: 0.1
9.37  - Min Value: 0.0134  - Mean Value: 0.1


In [18]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeConc[i].mean().item(),1))

Overall MAPE:  16.510633
--------------------
26.87  - Min Value: 0.008  - Mean Value: 0.8
31.55  - Min Value: 0.009  - Mean Value: 0.9
19.85  - Min Value: 0.0138  - Mean Value: 1.5
9.17  - Min Value: 0.0107  - Mean Value: 0.7
12.36  - Min Value: 0.0191  - Mean Value: 0.7
19.95  - Min Value: 0.0186  - Mean Value: 0.8
10.03  - Min Value: 0.0175  - Mean Value: 0.8
10.52  - Min Value: 0.0238  - Mean Value: 1.3
7.59  - Min Value: 0.0168  - Mean Value: 0.7
17.22  - Min Value: 0.0171  - Mean Value: 0.9


In [19]:
## Compute absolute percent error statistics on tissue mimicking distribution validation spectra (high relative glucose concentration)

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = MimicTissueRangeGlucConc[i]
    model_aq.eval()
    Prediction = model_aq(MimicTissueRangeGlucSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "MimicTissueRangeGlucExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(MimicTissueRangeGlucConc[i].min().item(),4), " - Mean Value:", np.round(MimicTissueRangeGlucConc[i].mean().item(),1))

Overall MAPE:  21.640154
--------------------
13.23  - Min Value: 0.013  - Mean Value: 0.6
25.14  - Min Value: 0.0115  - Mean Value: 0.4
25.64  - Min Value: 0.0115  - Mean Value: 0.4
16.74  - Min Value: 0.0115  - Mean Value: 0.6
17.58  - Min Value: 0.0115  - Mean Value: 1.0
23.8  - Min Value: 0.0115  - Mean Value: 1.1
31.01  - Min Value: 0.0115  - Mean Value: 0.8
16.34  - Min Value: 0.0115  - Mean Value: 0.5
21.78  - Min Value: 0.0115  - Mean Value: 0.5
25.14  - Min Value: 0.0115  - Mean Value: 1.1


In [20]:
## Compute absolute percent error statistics on a further high dynamic range dataset

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = HighDynamicRange2Conc[i]
    model_aq.eval()
    Prediction = model_aq(HighDynamicRange2Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth[metabolite] - Prediction_cpu[0][metabolite]) / GroundTruth[metabolite]
        APE.append(abs(per_err.cpu()))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "HighDynamicRange2Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "HighDynamicRange2Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2), " - Min Value:", np.round(HighDynamicRange2Conc[i].min().item(),4), " - Mean Value:", np.round(HighDynamicRange2Conc[i].mean().item(),1))

Overall MAPE:  134.71854
--------------------
99.82  - Min Value: 0.0062  - Mean Value: 2.1
111.96  - Min Value: 0.006  - Mean Value: 3.7
133.67  - Min Value: 0.0066  - Mean Value: 4.3
178.89  - Min Value: 0.0094  - Mean Value: 4.3
109.05  - Min Value: 0.0068  - Mean Value: 4.9
155.36  - Min Value: 0.005  - Mean Value: 3.8
95.55  - Min Value: 0.0101  - Mean Value: 3.2
55.72  - Min Value: 0.0062  - Mean Value: 3.2
145.89  - Min Value: 0.0053  - Mean Value: 5.3
261.27  - Min Value: 0.0054  - Mean Value: 2.5


In [21]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(SNR_Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "SNR_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "SNR_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  2.3969945387154747
--------------------
2.11
2.22
2.15
2.16
2.28
2.44
2.18
2.58
2.89
2.96


In [22]:
## Compute absolute percent error statistics on a dataset with singlets added at random

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(Singlet_Spec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "SingletExamples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "SingletExamples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  12.525569245250733
--------------------
2.18
2.56
2.87
4.11
13.62
13.79
14.41
21.39
22.87
27.45


In [23]:
## Compute absolute percent error statistics on a examples of varying SNR

APEs = []
MAPEs = []

for i in np.arange(10):
    GroundTruth = 0.43
    model_aq.eval()
    Prediction = model_aq(QrefSensSpec[i].unsqueeze(2))

    # Move Prediction tensor to CPU and detach from computation graph
    Prediction_cpu = Prediction.detach().cpu().numpy()

    APE = []

    for metabolite in range(44):
        per_err = 100*(GroundTruth - Prediction_cpu[0][metabolite]) / GroundTruth
        APE.append(abs(per_err))

    MAPE = sum(APE) / len(APE)

    APEs.append(APE)
    MAPEs.append(MAPE)


# Convert lists to numpy arrays and save
np.save(ModelName + "_" + "QrefSensitivity_Examples_APEs.npy", np.array(APEs))
np.save(ModelName + "_" + "QrefSensitivity_Examples_MAPEs.npy", np.array(MAPEs))



## Output metrics
print('Overall MAPE: ',np.array(MAPEs).mean())
print("--------------------")
for i in np.arange(10):
    print(round(MAPEs[i].item(), 2))

Overall MAPE:  144.4700195605622
--------------------
12.46
41.62
71.0
100.67
130.01
159.53
188.61
217.91
247.51
275.38


In [24]:
Pred = model_aq(OtherValSpectra[0].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 1")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[1].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("Sinusoidal Baseline 2")
print(Pred[0])
print("___________")
print("___________")

Pred = model_aq(OtherValSpectra[2].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("HD-Range 1 - 1s and 20s")
print(Pred[0])

Pred = model_aq(OtherValSpectra[3].unsqueeze(2))
Pred[0][Pred[0] < 0] = 0
print("HD-Range 2 - 0s and 20s")
print(Pred[0])

Sinusoidal Baseline 1
tensor([0.5796, 0.5505, 0.3089, 0.6632, 0.3929, 0.5110, 0.4809, 0.6571, 0.7394,
        0.3827, 0.3571, 0.6719, 0.5423, 0.9824, 0.6268, 0.6851, 0.7586, 0.4855,
        0.4968, 0.4251, 0.5102, 1.0338, 0.2188, 0.5248, 0.6128, 0.3797, 0.6085,
        0.6116, 0.5887, 0.4587, 0.4380, 0.9365, 0.5137, 0.4291, 0.3880, 0.5940,
        0.4802, 0.4597, 0.5299, 0.3477, 0.3555, 0.5050, 0.5482, 0.4477],
       device='cuda:0', grad_fn=<SelectBackward0>)
___________
___________
Sinusoidal Baseline 2
tensor([0.2734, 0.7213, 0.0000, 0.4910, 0.3141, 0.5442, 0.3806, 0.1756, 0.5411,
        0.6818, 0.4420, 0.5357, 0.9205, 0.1788, 0.7018, 0.0563, 0.7799, 0.5498,
        0.4844, 0.5038, 0.4401, 0.3982, 0.8114, 0.5956, 0.4900, 0.4528, 0.2288,
        0.0000, 0.4837, 0.2600, 0.3537, 0.2637, 0.0844, 0.5344, 0.6504, 0.5226,
        0.5774, 0.7775, 0.6230, 0.3288, 0.4006, 0.9498, 0.3296, 0.0853],
       device='cuda:0', grad_fn=<SelectBackward0>)
___________
___________
HD-Range 1 - 1s and 